# SuperAI Season 4 - Level 2 Hackathon - Forest Type Double Classifier

## Explore Data

In [23]:
import pandas as pd
import numpy as np

In [24]:
train_df = pd.read_csv('./datasets/train.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test.csv' , index_col='id')

## Create NON-DEF (MDF / DDF) / DEF 

In [25]:
DEF_df = train_df[train_df['nforest_type'] == 'DEF']
NON_DEF_df = train_df[train_df['nforest_type'] != 'DEF']

In [26]:
NON_DEF_df.to_csv('./datasets/MDF_DDF.csv')

In [34]:
NON_DEF_df['nforest_type'] = 'NON_DEF'
DEF_df['nforest_type'] = 'DEF'

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [35]:
NON_DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
2002,293,1927,1038,278,475,453,987,1773,2184,1900,2343,3039,NON_DEF
3212,197,1598,697,201,347,228,682,1982,2449,2254,2685,2690,NON_DEF
13312,929,1975,1031,982,1020,856,1220,2051,2421,2392,2671,2683,NON_DEF
17020,132,1560,689,189,408,175,609,2117,2907,3024,3005,2955,NON_DEF
5967,241,1944,1131,362,538,487,918,1549,1844,1702,2077,2043,NON_DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13977,1983,3602,2720,1622,1782,1766,2314,3488,3900,3924,4097,6053,NON_DEF
755,940,2007,1148,975,1080,968,1252,1780,1983,1942,2247,2170,NON_DEF
1616,1174,2312,1190,1112,1126,889,1310,2511,3085,3050,3396,3380,NON_DEF


In [36]:
DEF_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,b8_a,b9,nforest_type
id,,,,,,,,,,,,,
14932,701,1776,895,689,905,684,1187,2504,2836,2839,3172,3103,DEF
14572,156,1121,547,264,472,362,699,1673,1967,2182,2260,2277,DEF
7504,536,1591,777,686,760,624,1049,2152,2600,2665,2964,2777,DEF
10489,1551,1519,718,1730,1764,1514,1719,2267,2554,2598,2743,2743,DEF
7995,313,1696,864,366,568,411,849,1965,2387,2439,2827,2759,DEF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3762,182,1336,574,188,296,145,571,2000,2473,2243,2884,2844,DEF
7804,1,1753,820,106,431,301,840,2091,2472,2762,2914,2440,DEF
5566,281,1837,835,342,718,285,975,2801,3556,3637,3801,3148,DEF


In [37]:
DEF_NONDEF_df = pd.concat([NON_DEF_df , DEF_df])
DEF_NONDEF_df.sort_index().to_csv('./datasets/DEF_NONDEF.csv')

TypeError: '<' not supported between instances of 'int' and 'str'

## DEF_NONDEF Classifier

In [3]:
from autogluon.tabular import TabularPredictor
import pandas as pd
import numpy as np

In [4]:
DEF_NONDEF_df = pd.read_csv('./datasets/DEF_NONDEF.csv' , index_col='id')
test_df = pd.read_csv('./datasets/test.csv', index_col = 'id')

In [5]:
def add_features_for_DEF_NONDEF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row

def drop_features_for_DEF_NONDEF(df) :

    return df
    # return df.drop(columns = ['b1'])


In [7]:
DEF_NONDEF_df = DEF_NONDEF_df.apply(add_features_for_DEF_NONDEF , axis = 1)
test_df = test_df.apply(add_features_for_DEF_NONDEF , axis = 1)

DEF_NONDEF_df = drop_features_for_DEF_NONDEF(DEF_NONDEF_df)
test_df = drop_features_for_DEF_NONDEF(test_df)

C:\Users\teehe\AppData\Local\Temp\ipykernel_32292\269966182.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Normalized Difference 800/550'] = (b8 - b3) / (b8 + b3)
C:\Users\teehe\AppData\Local\Temp\ipykernel_32292\269966182.py:115: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Normalized Difference 800/650'] = (b8 - b4) / (b8 + b4)
C:\Users\teehe\AppData\Local\Temp\ipykernel_32292\269966182.py:116: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [8]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF         2585
Name: count, dtype: int64

In [9]:
from imblearn.over_sampling import SMOTE , KMeansSMOTE
from imblearn.combine import SMOTEENN , SMOTETomek

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

DEF_NONDEF_df , label_df  = smote.fit_resample(DEF_NONDEF_df.drop(columns=['nforest_type']) , DEF_NONDEF_df['nforest_type'])
DEF_NONDEF_df = DEF_NONDEF_df.join(label_df)

c:\anaconda\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
DEF_NONDEF_df['nforest_type'].value_counts()

nforest_type
NON_DEF    10468
DEF        10468
Name: count, dtype: int64

In [ ]:
label = 'nforest_type'
predictor = TabularPredictor(label = label).fit(DEF_NONDEF_df)

No path specified. Models will be saved in: "AutogluonModels\ag-20240605_090725"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240605_090725"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.0864374
[2000]	valid_set's binary_error: 0.0687679
[3000]	valid_set's binary_error: 0.0592168
[4000]	valid_set's binary_error: 0.0573066
[5000]	valid_set's binary_error: 0.056829
[6000]	valid_set's binary_error: 0.0563515


	0.9465	 = Validation score   (accuracy)
	5.37s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's binary_error: 0.0673352


	0.9389	 = Validation score   (accuracy)
	1.75s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9207	 = Validation score   (accuracy)
	2.44s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9226	 = Validation score   (accuracy)
	3.27s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.9451	 = Validation score   (accuracy)
	42.7s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9236	 = Validation score   (accuracy)
	0.82s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.9193	 = Validation score   (accuracy)
	0.78s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.9136	 = Validation score   (accuracy)
	12.25s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	0.9394	 = Validation score   (accuracy)
	2.52s	 = Training   runtime
	0.01s

[1000]	valid_set's binary_error: 0.0582617


	0.9446	 = Validation score   (accuracy)
	4.66s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.333, 'KNeighborsDist': 0.133, 'RandomForestGini': 0.133, 'CatBoost': 0.133, 'ExtraTreesGini': 0.133, 'LightGBMXT': 0.067, 'RandomForestEntr': 0.067}
	0.9599	 = Validation score   (accuracy)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 139.56s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240605_090725")


In [ ]:
predictor.refit_full()

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.03s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: KNeighborsDist_FULL ...
	0.02s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	3.85s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	1.29s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	2.57s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	3.62s	 = Training   runtime
Fitting 1 L1 models ...
Fitting mode

{'KNeighborsUnif': 'KNeighborsUnif_FULL',
 'KNeighborsDist': 'KNeighborsDist_FULL',
 'LightGBMXT': 'LightGBMXT_FULL',
 'LightGBM': 'LightGBM_FULL',
 'RandomForestGini': 'RandomForestGini_FULL',
 'RandomForestEntr': 'RandomForestEntr_FULL',
 'CatBoost': 'CatBoost_FULL',
 'ExtraTreesGini': 'ExtraTreesGini_FULL',
 'ExtraTreesEntr': 'ExtraTreesEntr_FULL',
 'NeuralNetFastAI': 'NeuralNetFastAI_FULL',
 'XGBoost': 'XGBoost_FULL',
 'NeuralNetTorch': 'NeuralNetTorch_FULL',
 'LightGBMLarge': 'LightGBMLarge_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}

In [ ]:
predictor.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 442 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.8983	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.9078	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0830946
[2000]	valid_set's binary_error: 0.0668577
[3000]	valid_set's binary_error: 0.0635148
[4000]	valid_set's binary_error: 0.0596944
[5000]	valid_set's binary_error: 0.0558739
[6000]	valid_set's binary_error: 0.0573066


	0.9451	 = Validation score   (accuracy)
	5.34s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.0639924
[2000]	valid_set's binary_error: 0.0530086
[3000]	valid_set's binary_error: 0.052531


	0.9499	 = Validation score   (accuracy)
	3.15s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.9202	 = Validation score   (accuracy)
	2.62s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.9222	 = Validation score   (accuracy)
	3.25s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.9475	 = Validation score   (accuracy)
	72.38s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.916	 = Validation score   (accuracy)
	0.87s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.9183	 = Validation score   (accuracy)
	1.15s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.9155	 = Validation score   (accuracy)
	13.23s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost_PSEUDO_1 ...
	0.9417	 = Va

[1000]	valid_set's binary_error: 0.056829


	0.9441	 = Validation score   (accuracy)
	4.51s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: WeightedEnsemble_L2_PSEUDO_1 ...
	Ensemble Weights: {'NeuralNetTorch_PSEUDO_1': 0.35, 'KNeighborsDist_PSEUDO_1': 0.25, 'ExtraTreesEntr_PSEUDO_1': 0.1, 'XGBoost_PSEUDO_1': 0.1, 'LightGBM_PSEUDO_1': 0.05, 'ExtraTreesGini_PSEUDO_1': 0.05, 'NeuralNetFastAI_PSEUDO_1': 0.05, 'LightGBMLarge_PSEUDO_1': 0.05}
	0.9618	 = Validation score   (accuracy)
	0.1s	 = Training   runtime
	0.0s	 = Validation runtime
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20240605_090725")
Pseudolabeling algorithm changed validation score from: 0.9598853868194842, to: 0.9598853868194842 using evaluation metric: accuracy


In [ ]:
prediction = predictor.predict(test_df)

In [ ]:
feature_importances = predictor.feature_importance(DEF_NONDEF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...
	154.81s	= Expected runtime (30.96s per shuffle set)
	47.88s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b11,0.24492,0.002610,1.547276e-09,5,0.250294,0.239546
b8_a,0.16952,0.001847,1.691402e-09,5,0.173323,0.165717
b9,0.11240,0.002973,5.869694e-08,5,0.118522,0.106278
b6,0.10120,0.004905,6.602240e-07,5,0.111300,0.091100
b12,0.09140,0.005464,1.525841e-06,5,0.102651,0.080149
b5,0.08508,0.000879,1.364722e-09,5,0.086889,0.083271
b7,0.07880,0.004007,7.999655e-07,5,0.087051,0.070549
b1,0.07224,0.003918,1.033886e-06,5,0.080307,0.064173
GRVI,0.06680,0.002173,1.340748e-07,5,0.071273,0.062327
b8,0.06344,0.004124,2.131042e-06,5,0.071932,0.054948


In [ ]:
feature_importances.to_csv('./features/DEF_NONDEF_features_importance.csv')

In [ ]:
prediction.value_counts()

nforest_type
NON_DEF    3011
DEF         989
Name: count, dtype: int64

In [ ]:
prediction = prediction.sort_index()

In [ ]:
prediction

id
1        NON_DEF
2        NON_DEF
5        NON_DEF
7        NON_DEF
12           DEF
          ...   
17039    NON_DEF
17042    NON_DEF
17043    NON_DEF
17047    NON_DEF
17052    NON_DEF
Name: nforest_type, Length: 4000, dtype: object

In [ ]:
prediction_DEF_ONLY = prediction[prediction == 'DEF']
prediction_DEF_ONLY

id
12       DEF
19       DEF
29       DEF
50       DEF
51       DEF
        ... 
17013    DEF
17024    DEF
17026    DEF
17032    DEF
17034    DEF
Name: nforest_type, Length: 989, dtype: object

In [ ]:
test_df = test_df.sort_index()

## MDF_DDF Classifier

In [ ]:
test_df = pd.read_csv('./datasets/test.csv', index_col='id')

In [ ]:
test_df = test_df[prediction == 'NON_DEF']

C:\Users\teehe\AppData\Local\Temp\ipykernel_24200\254591184.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = test_df[prediction == 'NON_DEF']


In [ ]:
MDF_DDF_df = pd.read_csv('./datasets/MDF_DDF.csv' , index_col='id')

In [ ]:
def add_features_for_MDF_DDF(row) :

    row['NDVI'] = (row['b8'] - row['b4']) / (row['b8'] + row['b4'])
    row['EVI'] = 2.5 * ((row['b8'] - row['b4']) / (row['b8'] + 6 * row['b4'] - 7.5 * row['b2'] + 1.01))
    row['NDWI '] = (row['b3'] - row['b8']) / (row['b3'] + row['b8'])
    row['SAVI '] = (row['b8'] - row['b4']) * (1 + 0.5) / (row['b8'] + row['b4'] + 0.5)
    row['MSAVI'] = (2 * row['b8'] + 1 - ( (2 * row['b8'] + 1) ** 2 - 8 * (row['b8'] - row['b4'])) ** (1 / 2)) / 2
    row['GNDVI '] = (row['b8'] - row['b3']) / (row['b8'] + row['b3'])
    row['RENDVI '] = (row['b8'] - row['b5']) / (row['b8'] + row['b5'])
    row['NDMI '] = (row['b8'] - row['b11']) / (row['b8'] + row['b11'])
    row['GRVI'] = (row['b3'] - row['b4']) / (row['b3'] + row['b4'])
    row['TVI'] = ( (row['b8'] - row['b4']) / (row['b8'] + row['b4'] + 0.5) ) ** (1 / 2)
    row['MCARI'] = ((row['b5'] - row['b4']) - 0.2 * (row['b5'] - row['b3'])) / (row['b5'] / row['b4'])
    row['BSI'] =  ((row['b11'] + row['b4']) - (row['b8'] + row['b2'])) / ((row['b11'] + row['b4']) + (row['b8'] + row['b2']))
    row['NBR'] = (row['b8'] - row['b12']) / (row['b8'] + row['b12'])
    row['MSI'] = row['b11'] / row['b8']

    return row

def drop_features_for_MDF_DDF(df) :

    return df
    # return df.drop(columns = ['b1'])

In [ ]:
MDF_DDF_df = MDF_DDF_df.apply(add_features_for_MDF_DDF , axis = 1)
MDF_DDF_df = drop_features_for_MDF_DDF(MDF_DDF_df)

test_df = test_df.apply(add_features_for_MDF_DDF , axis = 1)
test_df = drop_features_for_MDF_DDF(test_df)

In [ ]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    4603
Name: count, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE , KMeansSMOTE
from imblearn.combine import SMOTEENN , SMOTETomek

smote = SMOTE(random_state = 42 , sampling_strategy= 'all')

MDF_DDF_df , label_df  = smote.fit_resample(MDF_DDF_df.drop(columns=['nforest_type']) , MDF_DDF_df['nforest_type'])
MDF_DDF_df = MDF_DDF_df.join(label_df)

In [ ]:
MDF_DDF_df['nforest_type'].value_counts()

nforest_type
MDF    5865
DDF    5865
Name: count, dtype: int64

In [ ]:
predictor_for_MDF_DDF = TabularPredictor(label = label).fit(MDF_DDF_df)

No path specified. Models will be saved in: "AutogluonModels\ag-20240605_091623"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20240605_091623"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:      

[1000]	valid_set's binary_error: 0.244672


	0.7621	 = Validation score   (accuracy)
	1.34s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBM ...
	0.7681	 = Validation score   (accuracy)


[1000]	valid_set's binary_error: 0.236999


	1.11s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7613	 = Validation score   (accuracy)
	1.49s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7579	 = Validation score   (accuracy)
	2.15s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost ...
	0.7442	 = Validation score   (accuracy)
	5.53s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7656	 = Validation score   (accuracy)
	0.68s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7621	 = Validation score   (accuracy)
	0.77s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.7596	 = Validation score   (accuracy)
	6.89s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: XGBoost ...
	0.7613	 = Validation score   (accuracy)
	2.95s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: Neura

In [ ]:
predictor_for_MDF_DDF.refit_full()

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: KNeighborsUnif_FULL ...
	0.02s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: KNeighborsDist_FULL ...
	0.01s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBMXT_FULL ...
	1.1s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: LightGBM_FULL ...
	0.87s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestGini_FULL ...
	1.63s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model: RandomForestEntr_FULL ...
	2.4s	 = Training   runtime
Fitting 1 L1 models ...
Fitting model:

{'KNeighborsUnif': 'KNeighborsUnif_FULL',
 'KNeighborsDist': 'KNeighborsDist_FULL',
 'LightGBMXT': 'LightGBMXT_FULL',
 'LightGBM': 'LightGBM_FULL',
 'RandomForestGini': 'RandomForestGini_FULL',
 'RandomForestEntr': 'RandomForestEntr_FULL',
 'CatBoost': 'CatBoost_FULL',
 'ExtraTreesGini': 'ExtraTreesGini_FULL',
 'ExtraTreesEntr': 'ExtraTreesEntr_FULL',
 'NeuralNetFastAI': 'NeuralNetFastAI_FULL',
 'XGBoost': 'XGBoost_FULL',
 'NeuralNetTorch': 'NeuralNetTorch_FULL',
 'LightGBMLarge': 'LightGBMLarge_FULL',
 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}

In [ ]:
predictor_for_MDF_DDF.fit_pseudolabel(test_df)

Given test_data for pseudo labeling did not contain labels. AutoGluon will assign pseudo labels to data and use it for extra training data...
Beginning iteration 1 of pseudolabeling out of max 3
Pseudolabeling algorithm confidently assigned pseudolabels to 98 rows of data on iteration 1. Adding to train data
Fitting 13 L1 models ...
Fitting model: KNeighborsUnif_PSEUDO_1 ...
	0.7272	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: KNeighborsDist_PSEUDO_1 ...
	0.7425	 = Validation score   (accuracy)
	0.02s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: LightGBMXT_PSEUDO_1 ...


[1000]	valid_set's binary_error: 0.238704


	0.7681	 = Validation score   (accuracy)
	1.3s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_PSEUDO_1 ...
	0.7724	 = Validation score   (accuracy)
	0.89s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini_PSEUDO_1 ...
	0.763	 = Validation score   (accuracy)
	1.49s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr_PSEUDO_1 ...
	0.7579	 = Validation score   (accuracy)
	2.17s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: CatBoost_PSEUDO_1 ...
	0.7732	 = Validation score   (accuracy)
	29.29s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesGini_PSEUDO_1 ...
	0.7639	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: ExtraTreesEntr_PSEUDO_1 ...
	0.7613	 = Validation score   (accuracy)
	0.7s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetFastAI_PSEUDO_1 ...
	0.7613	 = Vali

In [ ]:
prediction_MDF_DDF = predictor_for_MDF_DDF.predict(add_features_for_MDF_DDF( test_df) )

In [ ]:
prediction_MDF

NameError: name 'prediction_MDF' is not defined

In [ ]:
feature_importances = predictor_for_MDF_DDF.feature_importance(MDF_DDF_df)
feature_importances

Computing feature importance via permutation shuffling for 26 features using 5000 rows with 5 shuffle sets...
	119.86s	= Expected runtime (23.97s per shuffle set)
	40.46s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
b9,0.28252,0.006830,4.096252e-08,5,0.296584,0.268456
b7,0.22984,0.006956,1.005597e-07,5,0.244163,0.215517
b8,0.19548,0.007180,2.180129e-07,5,0.210264,0.180696
b12,0.19240,0.004707,4.296887e-08,5,0.202093,0.182707
b11,0.18432,0.005509,9.566407e-08,5,0.195664,0.172976
b5,0.18248,0.003495,1.613175e-08,5,0.189675,0.175285
b8_a,0.18012,0.006836,2.485013e-07,5,0.194196,0.166044
b6,0.16544,0.005070,1.057334e-07,5,0.175880,0.155000
b1,0.13084,0.002286,1.118699e-08,5,0.135548,0.126132
b3,0.11756,0.001322,1.919343e-09,5,0.120282,0.114838


In [ ]:
feature_importances.to_csv('./features/MDF_DDF_features_importance.csv')

In [ ]:
prediction_MDF_DDF.value_counts()

nforest_type
DDF    1582
MDF    1419
Name: count, dtype: int64

In [ ]:
submission_df = pd.concat([prediction_DEF_ONLY , prediction_MDF_DDF])
submission_df = submission_df.sort_index()
submission_df

id
1        MDF
2        DDF
5        MDF
7        DDF
12       DEF
        ... 
17039    DDF
17042    DDF
17043    DDF
17047    MDF
17052    DDF
Name: nforest_type, Length: 4000, dtype: object

In [ ]:
submission_df.value_counts()

nforest_type
DDF    1582
MDF    1419
DEF     999
Name: count, dtype: int64

In [ ]:
submission_path = './submissions'
submission_df.to_csv(f'{submission_path}/submission_2over_features_smote_pseudolabeling_refit_double_classifier.csv')